In [1]:

# INDIR = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20"
# OUTDIR = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/results/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/clones_dendro"
# DONOR = 3
# COUNT_INDIR = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/donor3/mgatk"
# var_type = "mgatkdonor"

#INDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/"


cells_meta_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_mt_bestparams_af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0/clonalShift_method_mt_as_clones_dendro/cells_meta.tsv"
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_mt_bestparams_af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0/clonalShift_method_mt_as_clones_dendro/single/"

#cloneID = "0_C6"


In [2]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import mplh.fig_utils as fu

import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)

import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

fig_utils from mplh
0.5.6


In [3]:
import seaborn as sns
from mplh import cluster_help as ch

In [4]:
cells_meta = pd.read_csv(cells_meta_f,sep="\t", index_col=0)
cells_meta

,donor,cloneID,condition,clusterID
Input_GCGCCAAGTTGCCTGG-1,0,0_C8,Input,MPP/CMP/GMP
Flt3l_GGTCATACAAGGCGTA-1,0,0_C7,Flt3l,neutrophil B
Control_CAGTGCGAGGTTCTCA-1,0,0_C0,Control,eosonophil
Input_ACTTTCAGTACTATGC-1,0,0_C8,Input,MPP/CMP/GMP
Input_TTGACGATCCTCCATG-1,0,0_C8,Input,neutrophil B
...,...,...,...,...
Control_GGATAGGAGCTCGTTA-1,0,0_C1,Control,neutrophil
Input_ACATGCAAGAAATACC-1,0,0_C0,Input,MPP/CMP/GMP B
Input_TCCAGAAGTAGGGTCA-1,0,0_C5,Input,MPP/CMP/GMP B
Flt3l_CTGTTCGGTGGTTCTA-1,0,0_C0,Flt3l,meg/erythroid


## Filter for clone

In [5]:
norm_condition = cells_meta.groupby("condition").size()
norm_condition

norm_lineage = cells_meta.groupby("clusterID").size()
norm_lineage

clusterID
CLP               177
DC                199
MEP               310
MPP/CMP/GMP       968
MPP/CMP/GMP B    1353
MPP/CMP/GMP C     554
MPP/CMP/GMP D     182
eosonophil        692
erythroid         483
macrophage       1183
meg/erythroid     564
monocyte          382
neutrophil       1085
neutrophil B      487
dtype: int64

In [6]:
for clone_id, curr_cells in cells_meta.groupby("cloneID"):
    print(clone_id)

    curr_dir = join(outdir, f"cloneID_{clone_id}")
    if not exists(curr_dir):
        os.mkdir(curr_dir)
        
    f, ax = plt.subplots(figsize=(12,12), dpi=300)

    curr_cells_group = curr_cells.groupby(["clusterID", "condition"]).size().reset_index().pivot(index="clusterID", columns="condition", values=0).fillna(0)
    curr_cells_group.plot.bar(stacked=True, ax=ax)
    plt.savefig(join(curr_dir, "cluster_condition_ncells.png"))
    f, ax = plt.subplots(figsize=(12,12), dpi=300)
    curr_cells_group_norm = curr_cells_group.apply(lambda x: x/norm_condition.loc[x.name], axis=0)
    curr_cells_group_norm.plot.bar(stacked=True, ax=ax)
    plt.savefig(join(curr_dir, "cluster_condition_ncells_norm.png"))
    
    f, ax = plt.subplots(figsize=(12,12), dpi=300)
    curr_cells_lineage_group = curr_cells.groupby(["clusterID", "condition"]).size().reset_index().pivot(columns="clusterID", index="condition", values=0).fillna(0)
    curr_cells_group_norm = curr_cells_lineage_group.apply(lambda x: x/norm_lineage.loc[x.name], axis=0)
    curr_cells_lineage_group.plot.bar(stacked=True, ax=ax)
    plt.savefig(join(curr_dir, "cluster_lineage_splitCondition_ncells_norm.png"))
    plt.close('all')

0_C0
0_C1
0_C2
0_C3
0_C4
0_C5
0_C6
0_C7
0_C8
0_C9
